#  Assignment week 3 - Part 2

## Install beautifulsoup4

In [6]:
#!conda install --yes beautifulsoup4

## Libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## settings

In [3]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(wikiurl)
s = BeautifulSoup(response.text,'lxml')     
data = []

## find table

In [5]:
table = s.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

## extract values

In [6]:
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [e.text.strip() for e in cols]
    data.append([e for e in cols if e]) 

## apply rules

In [11]:
# convert into dataframe with specified columns
My_table = pd.DataFrame(data, columns = ['PostalCode','Burough','Neighborhood'])

# drop unassigned Buroughs
My_table.drop(My_table.index[0],inplace = True) # drop first empty row
My_table.drop(My_table[My_table['Burough']== 'Not assigned'].index, inplace = True)

# set Burough name to Unassigned Neighborhood
My_table.Neighborhood[My_table.Neighborhood == 'Not assigned'] = My_table.Burough[My_table.Neighborhood == 'Not assigned']
My_table = My_table.reset_index(drop = True)

# combine Neighborhoods with same postal-code
My_table2 = pd.DataFrame(My_table)
for i in My_table2.index:
        val = My_table2.at[i,'PostalCode']
        neigh = My_table2.at[i,'Neighborhood'] # get_value(i,'Neighborhood')
        neigh_prev = neigh
        prev_index = 0
        for j in My_table2.index:            
            if My_table2.at[j,'PostalCode'] == val and My_table2.at[j,'Neighborhood'] != neigh_prev:                 
                neigh = neigh + ', ' + My_table.at[j,'Neighborhood']
            elif My_table2.at[j,'PostalCode'] == val and My_table2.at[j,'Neighborhood']== neigh:
                neigh_prev = My_table2.at[j,'Neighborhood']
                neigh = My_table2.at[j,'Neighborhood']                   
           
        My_table2.at[i,'Neighborhood']=neigh
        
My_table2.drop_duplicates(subset='PostalCode', keep='first', inplace= True)        
My_table2.reset_index(drop = True)


PostalCode           Burough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
..         ...               ...   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Harbourfront, Regent Park  
3                     Lawrence Heights, Lawrence Manor  
4                                         Queen's Park  
5                                     Islington Avenue  
6                                       Rouge, Malvern  
7                                      Don Mills North  
8                      Woodbine Gardens, Parkview Hill  
9                             Ryerson, Garden District  
10                                           Glencairn  
11   Cloverdale, Islington, Martin Grove, Princess ...  
12              Highland Creek, Rouge Hill, Port Union  
13                    Flemingdon Park, Don Mills South  
14                                    Woodbine Heights  
15                                      St. James Town  
16                                  Humewood-Cedarvale  
17   Bloordale Gardens, Eringate, Markland Wood, Ol...  
18                   Guildwood, Morningside, West Hill  
19                                         The Beaches  
20                                         Berczy Park  
21                                 Caledonia-Fairbanks  
22                                              Woburn  
23                                             Leaside  
24                                  Central Bay Street  
25                                            Christie  
26                                           Cedarbrae  
27                                   Hillcrest Village  
28     Bathurst Manor, Downsview North, Wilson Heights  
29                                    Thorncliffe Park  
..  

In [12]:
My_table2.shape

(103, 3)

In [13]:
import requests
df_csv = pd.read_csv('GeospatialCoordinates.csv')

df_csv.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_csv.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [15]:
My_table2 = pd.merge(My_table2, df_csv, on='PostalCode')
My_table2

PostalCode           Burough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
..         ...               ...   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Harbourfront, Regent Park  43.654260 -79.360636  
3                     Lawrence Heights, Lawrence Manor  43.718518 -79.464763  
4                                         Queen's Park  43.662301 -79.389494  
5                                     Islington Avenue  43.667856 -79.532242  
6                                       Rouge, Malvern  43.806686 -79.194353  
7                                      Don Mills North  43.745906 -79.352188  
8                      Woodbine Gardens, Parkview Hill  43.706397 -79.309937  
9                             Ryerson, Garden District  43.657162 -79.378937  
10                                           Glencairn  43.709577 -79.445073  
11   Cloverdale, Islington, Martin Grove, Princess ...  43.650943 -79.554724  
12              Highland Creek, Rouge Hill, Port Union  43.784535 -79.160497  
13                    Flemingdon Park, Don Mills South  43.725900 -79.340923  
14                                    Woodbine Heights  43.695344 -79.318389  
15                                      St. James Town  43.651494 -79.375418  
16                                  Humewood-Cedarvale  43.693781 -79.428191  
17   Bloordale Gardens, Eringate, Markland Wood, Ol...  43.643515 -79.577201  
18                   Guildwood, Morningside, West Hill  43.763573 -79.188711  
19                                         The Beaches  43.676357 -79.293031  
20                                         Berczy Park  43.644771 -79.373306  
21                               